In [2]:
# 작업 폴더 설정
%cd /home/yoo/works/CGD

/home/yoo/works/CGD


In [ ]:
# data/custom 폴더 하위에 있는 이미지 파일(jpg, JPG)를 하나의 텍스트 파일에 정리

# import os

# search_path = 'data/custom'

# txt = open("{}/test.txt".format(search_path), 'w')
# txt.write("image_id\tclass_id\tsuper_class_id\tpath")

# idx = 0

# for (path, dir, files) in os.walk(search_path):

#     for filename in files:
#         ext = os.path.splitext(filename)[-1]

#         if path.split('/')[-1] != 'uncropped':
            
#             if ext in ['.jpg', '.JPG']:
#                 idx += 1
#                 txt.write("\n")
#                 txt.write(f"_\t{idx}\t_\t{os.path.join(path, filename)}")
#                 # txt.write(f"_\t{idx}\t_\t{filename}")

# txt.close()

In [18]:
# 불필요한 폴더를 제외하고 하위 폴더에 있는 이미지 파일 목록을 텍스트 파일로 저장
# 코드가 반복문을 이중으로 쓰고 있어서 효율화할 필요가 있음(!!!)

import os
import numpy as np

search_path = 'data/custom'
except_dirs = ['backup', 'uncropped']

with open(f"{search_path}/test.txt", 'w') as txt:
    txt.write("image_id\tclass_id\tsuper_class_id\tpath")
    
    classes = []
    files = []

    dirs = [dir for dir in os.listdir(search_path) if dir not in except_dirs and os.path.isdir(os.path.join(search_path, dir))]

    for dir in dirs:
        read_path = os.path.join(search_path, dir)
        for dp, dn, filenames in os.walk(read_path):
            for f in filenames:
                if f.endswith(('.jpg', '.JPG')):
                    files.append(os.path.join(dp, f))

    files.sort()

    class_ids = sorted(set(os.path.basename(f).split('_')[0] for f in files))

    for image_id, path in enumerate(files, 1):

        class_id = class_ids.index(os.path.basename(path).split('_')[0]) + 1

        txt.write("\n")
        txt.write(f"{image_id}\t{class_id}\t_\t{path}")

In [ ]:
def replace_in_file(file_path, old_str, new_str):

    # 파일 읽어들이기
    fr = open(file_path, 'r')
    lines = fr.readlines()
    fr.close()
    
    # old_str -> new_str 치환
    fw = open(file_path, 'w')
    for line in lines:
        fw.write(line.replace(old_str, new_str))
    fw.close()

# 호출: file1.txt 파일에서 comma(,) 없애기
replace_in_file("data/custom/test.txt", "bicycle_final", "data/custom/bicycle")
replace_in_file("data/custom/train.txt", "bicycle_final", "data/custom/bicycle")

In [ ]:
# 데이터베이스 생성 (uncropped_data_dicts.pth)

import data_utils

data_path = 'data/custom'

data_utils.process_custom_data(data_path)

In [ ]:
# 데이터베이스 확인

import torch

pth_path = 'data/custom/uncropped_data_dicts.pth'

data_base = torch.load(pth_path)

In [ ]:
data_base['train']['1']

In [ ]:
data_base['test']['11319']

In [2]:
# Train

!python train.py \
--data_path data \
--data_name custom \
--feature_dim 1536 \
--recalls '1,10,100' \
--batch_size 32

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
# Model Params: 27.83M FLOPs: 10.68G
Train Epoch 1/20 - Loss:6.5348 - Acc:0.75%: 100%|█| 130/130 [00:41<00:00,  3.11i
processing test data: 100%|███████████████████| 130/130 [00:14<00:00,  9.07it/s]
Test Epoch 1/20 R@1:21.69% R@10:49.14% R@100:79.03% 
Train Epoch 2/20 - Loss:6.4913 - Acc:0.67%: 100%|█| 130/130 [00:42<00:00,  3.08i
processing test data: 100%|███████████████████| 130/130 [00:14<00:00,  8.91it/s]
Test Epoch 2/20 R@

In [ ]:
# Inference 1
# 학습된 모델(model.pth)을 이용하여 다른 데이터의 feature map 파일(data_base.pth)을 생성

import argparse

import pandas as pd
import torch
from thop import profile, clever_format
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from tqdm import tqdm

from model import Model, set_bn_eval
from utils import recall, LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler

data_path = 'data'
data_name = 'custom'
crop_type = 'uncropped'
backbone_type = 'resnet50'
gd_config = 'SG'
feature_dim = 1536
smoothing = 0.1
temperature = 0.5
margin = 0.1
recalls = '1,2,4,8'
batch_size = 128

save_name_pre = '{}_{}_{}_{}_{}_{}_{}_{}_{}'.format(data_name, crop_type, backbone_type, gd_config, feature_dim,
                                                        smoothing, temperature, margin, batch_size)

test_data_set = ImageReader(data_path, data_name, 'test', crop_type)
test_data_loader = DataLoader(test_data_set, batch_size, shuffle=False, num_workers=8)
eval_dict = {'test': {'data_loader': test_data_loader}}

# model = Model(backbone_type, gd_config, feature_dim, num_classes=len(test_data_set.class_to_idx)).cuda()
model = Model(backbone_type, gd_config, feature_dim, num_classes=11318).cuda()

pretrained_model_path = 'results/sop_uncropped_resnet50_SG_1536_0.1_0.5_0.1_128_model.pth'
model.load_state_dict(torch.load(pretrained_model_path))

model.eval()
with torch.no_grad():
    # obtain feature vectors for all data
    for key in eval_dict.keys():
        eval_dict[key]['features'] = []
        for inputs, labels in tqdm(eval_dict[key]['data_loader'], desc='processing {} data'.format(key)):
            inputs, labels = inputs.cuda(), labels.cuda()
            features, classes = model(inputs)
            eval_dict[key]['features'].append(features)
        eval_dict[key]['features'] = torch.cat(eval_dict[key]['features'], dim=0)

data_base = {}
data_base['test_images'] = test_data_set.images
data_base['test_labels'] = test_data_set.labels
data_base['test_features'] = eval_dict['test']['features']
# torch.save(model.state_dict(), 'results/{}_model.pth'.format(save_name_pre))
torch.save(data_base, 'results/{}_data_base.pth'.format(save_name_pre))

In [ ]:
# Inference 2

!python test.py \
--query_img_name 'data/custom/uncropped/251952414262_0.JPG' \
--data_base 'results/custom_uncropped_resnet50_SG_1536_0.1_0.5_0.1_32_data_base.pth' \
--retrieval_num 10